In [4]:
import numpy as np
import pandas as pd

In [250]:
Dvector_devset = pd.read_csv('/home/bibash_01/my_project/fearless/fearless/diary/onehot_devset.scp', sep = '  ', header = None, names=['ID_details', 'path'])

/net/software/python/2020_06/anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [251]:
Dvector_devset

,ID_details,path
0,AFD1-FS02_ASR_track2_dev_0144,/net/vol/bibash_01/experiments/fearless/asr2/d...
1,AFD1-FS02_ASR_track2_dev_0282,/net/vol/bibash_01/experiments/fearless/asr2/d...
2,AFD1-FS02_ASR_track2_dev_0670,/net/vol/bibash_01/experiments/fearless/asr2/d...
3,AFD1-FS02_ASR_track2_dev_0716,/net/vol/bibash_01/experiments/fearless/asr2/d...
4,AFD1-FS02_ASR_track2_dev_1107,/net/vol/bibash_01/experiments/fearless/asr2/d...
...,...,...
8457,VOICE-FS02_ASR_track2_dev_6624,/net/vol/bibash_01/experiments/fearless/asr2/d...
8458,VOICE-FS02_ASR_track2_dev_6751,/net/vol/bibash_01/experiments/fearless/asr2/d...
8459,VOICE-FS02_ASR_track2_dev_7578,/net/vol/bibash_01/experiments/fearless/asr2/d...
8460,VOICE-FS02_ASR_track2_dev_8347,/net/vol/bibash_01/experiments/fearless/asr2/d...


In [252]:
utt2spk_devset = pd.read_csv('/home/bibash_01/my_project/fearless/fearless/diary/utt2spk_devset', sep = ' ', header = None, names=['ID_details', 'Speaker_name'])

In [253]:
utt2spk_devset

,ID_details,Speaker_name
0,AFD1-FS02_ASR_track2_dev_0144,AFD1
1,AFD1-FS02_ASR_track2_dev_0282,AFD1
2,AFD1-FS02_ASR_track2_dev_0670,AFD1
3,AFD1-FS02_ASR_track2_dev_0716,AFD1
4,AFD1-FS02_ASR_track2_dev_1107,AFD1
...,...,...
8457,VOICE-FS02_ASR_track2_dev_6624,VOICE
8458,VOICE-FS02_ASR_track2_dev_6751,VOICE
8459,VOICE-FS02_ASR_track2_dev_7578,VOICE
8460,VOICE-FS02_ASR_track2_dev_8347,VOICE


In [255]:
devset = pd.merge(Dvector_devset, utt2spk_devset, on=["ID_details"])
devset

,ID_details,path,Speaker_name
0,AFD1-FS02_ASR_track2_dev_0144,/net/vol/bibash_01/experiments/fearless/asr2/d...,AFD1
1,AFD1-FS02_ASR_track2_dev_0282,/net/vol/bibash_01/experiments/fearless/asr2/d...,AFD1
2,AFD1-FS02_ASR_track2_dev_0670,/net/vol/bibash_01/experiments/fearless/asr2/d...,AFD1
3,AFD1-FS02_ASR_track2_dev_0716,/net/vol/bibash_01/experiments/fearless/asr2/d...,AFD1
4,AFD1-FS02_ASR_track2_dev_1107,/net/vol/bibash_01/experiments/fearless/asr2/d...,AFD1
...,...,...,...
8457,VOICE-FS02_ASR_track2_dev_6624,/net/vol/bibash_01/experiments/fearless/asr2/d...,VOICE
8458,VOICE-FS02_ASR_track2_dev_6751,/net/vol/bibash_01/experiments/fearless/asr2/d...,VOICE
8459,VOICE-FS02_ASR_track2_dev_7578,/net/vol/bibash_01/experiments/fearless/asr2/d...,VOICE
8460,VOICE-FS02_ASR_track2_dev_8347,/net/vol/bibash_01/experiments/fearless/asr2/d...,VOICE


### Hints from Thilo
-> You pick pairs of vectors, ideally roughly the same number of "positive" (same speaker) and "negative" (different speaker) pairs. Make sure you use the same pairs every time you compute the EER, otherwise they are not comparable.

->You compute the true label for each pair: Either a one ("postivie") or a zero ("negative")
You compute the similarity between the vectors in each pair. You obtain one scalar number for each pair, between 0 and 1.

->You then compute the estimated label (0 or 1) of the speaker pairs. Because you don't know the "margin", you have to sweep through a threshold above which you decide for 1 and below which you decide for 0. The margin is chosen so that the false positive rate and the false negative rate (comparing the true and estimated labels) are equal (or, if not possible, at least as close as possible) -- thus its name "equal" error rate

->The error rate that you obtain is the EER (EER = false postivie rate = false negative rate).

In [256]:
#Unique nameset
unique_id = sorted(list(set(devset['Speaker_name'])), key=lambda name:name.lower())
len(unique_id)

166

In [257]:
devset_SFL = pd.merge(devset, devset, on='Speaker_name', suffixes=['','SFL']).query("ID_details != ID_detailsSFL").reset_index(drop=True)

In [264]:
devset_SFL

,ID_details,path,Speaker_name,ID_detailsSFL,pathSFL
0,AFD1-FS02_ASR_track2_dev_0144,/net/vol/bibash_01/experiments/fearless/asr2/d...,AFD1,AFD1-FS02_ASR_track2_dev_0282,/net/vol/bibash_01/experiments/fearless/asr2/d...
1,AFD1-FS02_ASR_track2_dev_0144,/net/vol/bibash_01/experiments/fearless/asr2/d...,AFD1,AFD1-FS02_ASR_track2_dev_0670,/net/vol/bibash_01/experiments/fearless/asr2/d...
2,AFD1-FS02_ASR_track2_dev_0144,/net/vol/bibash_01/experiments/fearless/asr2/d...,AFD1,AFD1-FS02_ASR_track2_dev_0716,/net/vol/bibash_01/experiments/fearless/asr2/d...
3,AFD1-FS02_ASR_track2_dev_0144,/net/vol/bibash_01/experiments/fearless/asr2/d...,AFD1,AFD1-FS02_ASR_track2_dev_1107,/net/vol/bibash_01/experiments/fearless/asr2/d...
4,AFD1-FS02_ASR_track2_dev_0144,/net/vol/bibash_01/experiments/fearless/asr2/d...,AFD1,AFD1-FS02_ASR_track2_dev_1516,/net/vol/bibash_01/experiments/fearless/asr2/d...
...,...,...,...,...,...
2525619,VOICE-FS02_ASR_track2_dev_8781,/net/vol/bibash_01/experiments/fearless/asr2/d...,VOICE,VOICE-FS02_ASR_track2_dev_6560,/net/vol/bibash_01/experiments/fearless/asr2/d...
2525620,VOICE-FS02_ASR_track2_dev_8781,/net/vol/bibash_01/experiments/fearless/asr2/d...,VOICE,VOICE-FS02_ASR_track2_dev_6624,/net/vol/bibash_01/experiments/fearless/asr2/d...
2525621,VOICE-FS02_ASR_track2_dev_8781,/net/vol/bibash_01/experiments/fearless/asr2/d...,VOICE,VOICE-FS02_ASR_track2_dev_6751,/net/vol/bibash_01/experiments/fearless/asr2/d...
2525622,VOICE-FS02_ASR_track2_dev_8781,/net/vol/bibash_01/experiments/fearless/asr2/d...,VOICE,VOICE-FS02_ASR_track2_dev_7578,/net/vol/bibash_01/experiments/fearless/asr2/d...


In [265]:
devset_SFL['pathSFL'][0]

'/net/vol/bibash_01/experiments/fearless/asr2/dump/raw/devset/embedding_vectors/AFD1-FS02_ASR_track2_dev_0282.npy'

In [266]:
devset_SFL['path'][1]

'/net/vol/bibash_01/experiments/fearless/asr2/dump/raw/devset/embedding_vectors/AFD1-FS02_ASR_track2_dev_0144.npy'

In [267]:
#devsegment
from tqdm import tqdm

In [268]:
# drop some samples

In [269]:
size = 100        # sample size
replace = True  # with replacement
fn = lambda obj: obj.loc[np.random.choice(obj.index, size, replace),:]
devset_SFL_filtered = devset_SFL.groupby('Speaker_name', as_index=False).apply(fn).reset_index(drop=True)

In [270]:
devset_SFL_filtered

,ID_details,path,Speaker_name,ID_detailsSFL,pathSFL
0,AFD1-FS02_ASR_track2_dev_4538,/net/vol/bibash_01/experiments/fearless/asr2/d...,AFD1,AFD1-FS02_ASR_track2_dev_4318,/net/vol/bibash_01/experiments/fearless/asr2/d...
1,AFD1-FS02_ASR_track2_dev_2025,/net/vol/bibash_01/experiments/fearless/asr2/d...,AFD1,AFD1-FS02_ASR_track2_dev_0670,/net/vol/bibash_01/experiments/fearless/asr2/d...
2,AFD1-FS02_ASR_track2_dev_0282,/net/vol/bibash_01/experiments/fearless/asr2/d...,AFD1,AFD1-FS02_ASR_track2_dev_1516,/net/vol/bibash_01/experiments/fearless/asr2/d...
3,AFD1-FS02_ASR_track2_dev_0282,/net/vol/bibash_01/experiments/fearless/asr2/d...,AFD1,AFD1-FS02_ASR_track2_dev_1516,/net/vol/bibash_01/experiments/fearless/asr2/d...
4,AFD1-FS02_ASR_track2_dev_7827,/net/vol/bibash_01/experiments/fearless/asr2/d...,AFD1,AFD1-FS02_ASR_track2_dev_8854,/net/vol/bibash_01/experiments/fearless/asr2/d...
...,...,...,...,...,...
15195,VOICE-FS02_ASR_track2_dev_6092,/net/vol/bibash_01/experiments/fearless/asr2/d...,VOICE,VOICE-FS02_ASR_track2_dev_6367,/net/vol/bibash_01/experiments/fearless/asr2/d...
15196,VOICE-FS02_ASR_track2_dev_4502,/net/vol/bibash_01/experiments/fearless/asr2/d...,VOICE,VOICE-FS02_ASR_track2_dev_6367,/net/vol/bibash_01/experiments/fearless/asr2/d...
15197,VOICE-FS02_ASR_track2_dev_4502,/net/vol/bibash_01/experiments/fearless/asr2/d...,VOICE,VOICE-FS02_ASR_track2_dev_1938,/net/vol/bibash_01/experiments/fearless/asr2/d...
15198,VOICE-FS02_ASR_track2_dev_5719,/net/vol/bibash_01/experiments/fearless/asr2/d...,VOICE,VOICE-FS02_ASR_track2_dev_0698,/net/vol/bibash_01/experiments/fearless/asr2/d...


In [271]:
#Negative Pairs:

In [275]:
#Unique nameset
unique_id = sorted(list(set(devset_SFL_filtered['Speaker_name'])), key=lambda name:name.lower())
len(unique_id)
#here unique_name 152 only, beacause of dropped operation.

152

In [276]:
#Negative Pairs
#AFD1

In [277]:
id1, id2 = unique_id[0], unique_id[1]
for i in range(len(devset_SFL_filtered)):
    count = sum(devset_SFL_filtered['Speaker_name']==id1)
    if (i + count) <= 15200:
        negativePair = (devset_SFL_filtered['ID_details'][i], devset_SFL_filtered['ID_details'][i+count])
        print(negativePair)
    else:
        break
   

('AFD1-FS02_ASR_track2_dev_4538', 'AFD5-FS02_ASR_track2_dev_5956')
('AFD1-FS02_ASR_track2_dev_2025', 'AFD5-FS02_ASR_track2_dev_6244')
('AFD1-FS02_ASR_track2_dev_0282', 'AFD5-FS02_ASR_track2_dev_5956')
('AFD1-FS02_ASR_track2_dev_0282', 'AFD5-FS02_ASR_track2_dev_2514')
('AFD1-FS02_ASR_track2_dev_7827', 'AFD5-FS02_ASR_track2_dev_4454')
('AFD1-FS02_ASR_track2_dev_0670', 'AFD5-FS02_ASR_track2_dev_2514')
('AFD1-FS02_ASR_track2_dev_4634', 'AFD5-FS02_ASR_track2_dev_2514')
('AFD1-FS02_ASR_track2_dev_0716', 'AFD5-FS02_ASR_track2_dev_7296')
('AFD1-FS02_ASR_track2_dev_8854', 'AFD5-FS02_ASR_track2_dev_7296')
('AFD1-FS02_ASR_track2_dev_8854', 'AFD5-FS02_ASR_track2_dev_5956')
('AFD1-FS02_ASR_track2_dev_1516', 'AFD5-FS02_ASR_track2_dev_1131')
('AFD1-FS02_ASR_track2_dev_3990', 'AFD5-FS02_ASR_track2_dev_5433')
('AFD1-FS02_ASR_track2_dev_1516', 'AFD5-FS02_ASR_track2_dev_7531')
('AFD1-FS02_ASR_track2_dev_8805', 'AFD5-FS02_ASR_track2_dev_7296')
('AFD1-FS02_ASR_track2_dev_0144', 'AFD5-FS02_ASR_track2_dev_44

('AFD5-FS02_ASR_track2_dev_1131', 'AGC-FS02_ASR_track2_dev_1117')
('AFD5-FS02_ASR_track2_dev_6244', 'AGC-FS02_ASR_track2_dev_3260')
('AFD5-FS02_ASR_track2_dev_7531', 'AGC-FS02_ASR_track2_dev_3260')
('AFD5-FS02_ASR_track2_dev_2514', 'AGC-FS02_ASR_track2_dev_1117')
('AFD5-FS02_ASR_track2_dev_5433', 'AGC-FS02_ASR_track2_dev_3260')
('AFD5-FS02_ASR_track2_dev_5956', 'AGC-FS02_ASR_track2_dev_3260')
('AFD5-FS02_ASR_track2_dev_7531', 'AGC-FS02_ASR_track2_dev_1117')
('AFD5-FS02_ASR_track2_dev_7531', 'AGC-FS02_ASR_track2_dev_1117')
('AFD5-FS02_ASR_track2_dev_7296', 'AGC-FS02_ASR_track2_dev_1117')
('AFD5-FS02_ASR_track2_dev_7531', 'AGC-FS02_ASR_track2_dev_1117')
('AFD5-FS02_ASR_track2_dev_5956', 'AGC-FS02_ASR_track2_dev_1117')
('AFD5-FS02_ASR_track2_dev_0140', 'AGC-FS02_ASR_track2_dev_1117')
('AFD5-FS02_ASR_track2_dev_0140', 'AGC-FS02_ASR_track2_dev_1117')
('AFD5-FS02_ASR_track2_dev_6244', 'AGC-FS02_ASR_track2_dev_3260')
('AFD5-FS02_ASR_track2_dev_5433', 'AGC-FS02_ASR_track2_dev_3260')
('AFD5-FS0

('BERMUDA-FS02_ASR_track2_dev_2099', 'BOOSTER1-FS02_ASR_track2_dev_0871')
('BERMUDA-FS02_ASR_track2_dev_2099', 'BOOSTER1-FS02_ASR_track2_dev_5477')
('BERMUDA-FS02_ASR_track2_dev_2099', 'BOOSTER1-FS02_ASR_track2_dev_5477')
('BERMUDA-FS02_ASR_track2_dev_7893', 'BOOSTER1-FS02_ASR_track2_dev_0775')
('BERMUDA-FS02_ASR_track2_dev_2099', 'BOOSTER1-FS02_ASR_track2_dev_0775')
('BERMUDA-FS02_ASR_track2_dev_2099', 'BOOSTER1-FS02_ASR_track2_dev_4509')
('BERMUDA-FS02_ASR_track2_dev_7893', 'BOOSTER1-FS02_ASR_track2_dev_4324')
('BERMUDA-FS02_ASR_track2_dev_7893', 'BOOSTER1-FS02_ASR_track2_dev_2111')
('BERMUDA-FS02_ASR_track2_dev_2099', 'BOOSTER1-FS02_ASR_track2_dev_2109')
('BERMUDA-FS02_ASR_track2_dev_7893', 'BOOSTER1-FS02_ASR_track2_dev_0775')
('BERMUDA-FS02_ASR_track2_dev_2099', 'BOOSTER1-FS02_ASR_track2_dev_6916')
('BERMUDA-FS02_ASR_track2_dev_2099', 'BOOSTER1-FS02_ASR_track2_dev_0775')
('BERMUDA-FS02_ASR_track2_dev_7893', 'BOOSTER1-FS02_ASR_track2_dev_2111')
('BERMUDA-FS02_ASR_track2_dev_2099', '

('COLLINS-FS02_ASR_track2_dev_0818', 'COMM-FS02_ASR_track2_dev_0498')
('COLLINS-FS02_ASR_track2_dev_6635', 'COMM-FS02_ASR_track2_dev_6638')
('COLLINS-FS02_ASR_track2_dev_3013', 'COMM-FS02_ASR_track2_dev_0498')
('COLLINS-FS02_ASR_track2_dev_4114', 'COMM-FS02_ASR_track2_dev_3343')
('COLLINS-FS02_ASR_track2_dev_1717', 'COMM-FS02_ASR_track2_dev_9078')
('COLLINS-FS02_ASR_track2_dev_3603', 'COMM-FS02_ASR_track2_dev_5334')
('COLLINS-FS02_ASR_track2_dev_5023', 'COMM-FS02_ASR_track2_dev_0498')
('COLLINS-FS02_ASR_track2_dev_5554', 'COMM-FS02_ASR_track2_dev_6638')
('COLLINS-FS02_ASR_track2_dev_6596', 'COMM-FS02_ASR_track2_dev_3343')
('COLLINS-FS02_ASR_track2_dev_7997', 'COMM-FS02_ASR_track2_dev_3343')
('COLLINS-FS02_ASR_track2_dev_3174', 'COMM-FS02_ASR_track2_dev_0456')
('COLLINS-FS02_ASR_track2_dev_1541', 'COMM-FS02_ASR_track2_dev_0456')
('COLLINS-FS02_ASR_track2_dev_6392', 'COMM-FS02_ASR_track2_dev_6638')
('COLLINS-FS02_ASR_track2_dev_8473', 'COMM-FS02_ASR_track2_dev_0456')
('COLLINS-FS02_ASR_t

('COMMCONTROL2-FS02_ASR_track2_dev_7895', 'COMMCONTROL3-FS02_ASR_track2_dev_5925')
('COMMCONTROL2-FS02_ASR_track2_dev_6737', 'COMMCONTROL3-FS02_ASR_track2_dev_6670')
('COMMCONTROL2-FS02_ASR_track2_dev_7895', 'COMMCONTROL3-FS02_ASR_track2_dev_5490')
('COMMCONTROL2-FS02_ASR_track2_dev_9130', 'COMMCONTROL3-FS02_ASR_track2_dev_7377')
('COMMCONTROL2-FS02_ASR_track2_dev_4675', 'COMMCONTROL3-FS02_ASR_track2_dev_8882')
('COMMCONTROL2-FS02_ASR_track2_dev_4675', 'COMMCONTROL3-FS02_ASR_track2_dev_8047')
('COMMCONTROL2-FS02_ASR_track2_dev_4181', 'COMMCONTROL3-FS02_ASR_track2_dev_6857')
('COMMCONTROL2-FS02_ASR_track2_dev_4675', 'COMMCONTROL3-FS02_ASR_track2_dev_3310')
('COMMCONTROL2-FS02_ASR_track2_dev_2284', 'COMMCONTROL3-FS02_ASR_track2_dev_4484')
('COMMCONTROL2-FS02_ASR_track2_dev_4181', 'COMMCONTROL3-FS02_ASR_track2_dev_2880')
('COMMCONTROL2-FS02_ASR_track2_dev_7913', 'COMMCONTROL3-FS02_ASR_track2_dev_0225')
('COMMCONTROL2-FS02_ASR_track2_dev_7806', 'COMMCONTROL3-FS02_ASR_track2_dev_5320')
('CO

...

('TIC1-FS02_ASR_track2_dev_0158', 'TIC2-FS02_ASR_track2_dev_4055')
('TIC1-FS02_ASR_track2_dev_7041', 'TIC2-FS02_ASR_track2_dev_8474')
('TIC1-FS02_ASR_track2_dev_0887', 'TIC2-FS02_ASR_track2_dev_3320')
('TIC1-FS02_ASR_track2_dev_1144', 'TIC2-FS02_ASR_track2_dev_3094')
('TIC1-FS02_ASR_track2_dev_1500', 'TIC2-FS02_ASR_track2_dev_2455')
('TIC1-FS02_ASR_track2_dev_0719', 'TIC2-FS02_ASR_track2_dev_3815')
('TIC1-FS02_ASR_track2_dev_5321', 'TIC2-FS02_ASR_track2_dev_8474')
('TIC1-FS02_ASR_track2_dev_0098', 'TIC2-FS02_ASR_track2_dev_5264')
('TIC1-FS02_ASR_track2_dev_6490', 'TIC2-FS02_ASR_track2_dev_3827')
('TIC1-FS02_ASR_track2_dev_1063', 'TIC2-FS02_ASR_track2_dev_5993')
('TIC1-FS02_ASR_track2_dev_4438', 'TIC2-FS02_ASR_track2_dev_8939')
('TIC1-FS02_ASR_track2_dev_2479', 'TIC2-FS02_ASR_track2_dev_2686')
('TIC1-FS02_ASR_track2_dev_1686', 'TIC2-FS02_ASR_track2_dev_9141')
('TIC1-FS02_ASR_track2_dev_2118', 'TIC2-FS02_ASR_track2_dev_1669')
('TIC1-FS02_ASR_track2_dev_2632', 'TIC2-FS02_ASR_track2_dev_04

('TIC2-FS02_ASR_track2_dev_3320', 'TIC5-FS02_ASR_track2_dev_6883')
('TIC2-FS02_ASR_track2_dev_4547', 'TIC5-FS02_ASR_track2_dev_4141')
('TIC2-FS02_ASR_track2_dev_5993', 'TIC5-FS02_ASR_track2_dev_3283')
('TIC2-FS02_ASR_track2_dev_9141', 'TIC5-FS02_ASR_track2_dev_1503')
('TIC2-FS02_ASR_track2_dev_9199', 'TIC5-FS02_ASR_track2_dev_1503')
('TIC5-FS02_ASR_track2_dev_3206', 'TRACK0-FS02_ASR_track2_dev_0454')
('TIC5-FS02_ASR_track2_dev_3283', 'TRACK0-FS02_ASR_track2_dev_5937')
('TIC5-FS02_ASR_track2_dev_3234', 'TRACK0-FS02_ASR_track2_dev_7630')
('TIC5-FS02_ASR_track2_dev_3234', 'TRACK0-FS02_ASR_track2_dev_0608')
('TIC5-FS02_ASR_track2_dev_4141', 'TRACK0-FS02_ASR_track2_dev_7379')
('TIC5-FS02_ASR_track2_dev_4141', 'TRACK0-FS02_ASR_track2_dev_4137')
('TIC5-FS02_ASR_track2_dev_6883', 'TRACK0-FS02_ASR_track2_dev_7630')
('TIC5-FS02_ASR_track2_dev_3206', 'TRACK0-FS02_ASR_track2_dev_2925')
('TIC5-FS02_ASR_track2_dev_4141', 'TRACK0-FS02_ASR_track2_dev_4521')
('TIC5-FS02_ASR_track2_dev_3257', 'TRACK0-FS

('TRACK0-FS02_ASR_track2_dev_0454', 'TRACK1-FS02_ASR_track2_dev_7116')
('TRACK0-FS02_ASR_track2_dev_0454', 'TRACK1-FS02_ASR_track2_dev_5103')
('TRACK0-FS02_ASR_track2_dev_4973', 'TRACK1-FS02_ASR_track2_dev_6198')
('TRACK0-FS02_ASR_track2_dev_4242', 'TRACK1-FS02_ASR_track2_dev_7116')
('TRACK0-FS02_ASR_track2_dev_0015', 'TRACK1-FS02_ASR_track2_dev_3253')
('TRACK0-FS02_ASR_track2_dev_0015', 'TRACK1-FS02_ASR_track2_dev_4562')
('TRACK0-FS02_ASR_track2_dev_0608', 'TRACK1-FS02_ASR_track2_dev_7116')
('TRACK0-FS02_ASR_track2_dev_1576', 'TRACK1-FS02_ASR_track2_dev_7305')
('TRACK0-FS02_ASR_track2_dev_3192', 'TRACK1-FS02_ASR_track2_dev_6198')
('TRACK0-FS02_ASR_track2_dev_0608', 'TRACK1-FS02_ASR_track2_dev_5103')
('TRACK0-FS02_ASR_track2_dev_4521', 'TRACK1-FS02_ASR_track2_dev_5782')
('TRACK0-FS02_ASR_track2_dev_2082', 'TRACK1-FS02_ASR_track2_dev_7455')
('TRACK0-FS02_ASR_track2_dev_5937', 'TRACK1-FS02_ASR_track2_dev_7305')
('TRACK0-FS02_ASR_track2_dev_7379', 'TRACK1-FS02_ASR_track2_dev_7550')
('TRAC

('UNK-FS02_ASR_track2_dev_3791', 'VANGUARD-FS02_ASR_track2_dev_8506')
('UNK-FS02_ASR_track2_dev_2174', 'VANGUARD-FS02_ASR_track2_dev_0681')
('UNK-FS02_ASR_track2_dev_3920', 'VANGUARD-FS02_ASR_track2_dev_8506')
('UNK-FS02_ASR_track2_dev_3920', 'VANGUARD-FS02_ASR_track2_dev_4309')
('UNK-FS02_ASR_track2_dev_6700', 'VANGUARD-FS02_ASR_track2_dev_0606')
('UNK-FS02_ASR_track2_dev_3188', 'VANGUARD-FS02_ASR_track2_dev_6556')
('UNK-FS02_ASR_track2_dev_2327', 'VANGUARD-FS02_ASR_track2_dev_2773')
('UNK-FS02_ASR_track2_dev_6566', 'VANGUARD-FS02_ASR_track2_dev_2773')
('UNK-FS02_ASR_track2_dev_8829', 'VANGUARD-FS02_ASR_track2_dev_4309')
('UNK-FS02_ASR_track2_dev_4432', 'VANGUARD-FS02_ASR_track2_dev_0681')
('UNK-FS02_ASR_track2_dev_4074', 'VANGUARD-FS02_ASR_track2_dev_4309')
('UNK-FS02_ASR_track2_dev_2650', 'VANGUARD-FS02_ASR_track2_dev_0606')
('UNK-FS02_ASR_track2_dev_1848', 'VANGUARD-FS02_ASR_track2_dev_6556')
('UNK-FS02_ASR_track2_dev_1391', 'VANGUARD-FS02_ASR_track2_dev_2773')
('UNK-FS02_ASR_track

KeyError: 15200

In [153]:
#Negative Pair
#AFD1

In [279]:
negative_pairs = [(Id1, Id2) for Id1 in unique_id for Id2 in unique_id if Id1 != Id2]
negative_pairs

[('AFD1', 'AFD5'),
 ('AFD1', 'AGC'),
 ('AFD1', 'ALDS'),
 ('AFD1', 'ALSEP'),
 ('AFD1', 'ARIA'),
 ('AFD1', 'ASCENSION'),
 ('AFD1', 'ASCENSIONCOMMTECH'),
 ('AFD1', 'BERMUDA'),
 ('AFD1', 'BOOSTER1'),
 ('AFD1', 'BUZZ'),
 ('AFD1', 'CANARY'),
 ('AFD1', 'CAPCOM1'),
 ('AFD1', 'CAPCOM2'),
 ('AFD1', 'CCATSCOMMAND'),
 ('AFD1', 'CCATSTELEMETRY'),
 ('AFD1', 'CLTC'),
 ('AFD1', 'COLLINS'),
 ('AFD1', 'COMM'),
 ('AFD1', 'COMMANAGER1'),
 ('AFD1', 'COMMCONTROL1'),
 ('AFD1', 'COMMCONTROL2'),
 ('AFD1', 'COMMCONTROL3'),
 ('AFD1', 'COMMCONTROL4'),
 ('AFD1', 'COMPUTERMNO'),
 ('AFD1', 'CONSUMABLES'),
 ('AFD1', 'CONTROL1'),
 ('AFD1', 'CONTROL2'),
 ('AFD1', 'CPSPK'),
 ('AFD1', 'CPSS'),
 ('AFD1', 'CTSC'),
 ('AFD1', 'CVTS'),
 ('AFD1', 'DATACORE'),
 ('AFD1', 'DEPUTYLOM'),
 ('AFD1', 'DISPLAY'),
 ('AFD1', 'ECS1'),
 ('AFD1', 'ECS2'),
 ('AFD1', 'EECOM1'),
 ('AFD1', 'EECOM2'),
 ('AFD1', 'EECOM3'),
 ('AFD1', 'EECOM5'),
 ('AFD1', 'EMU'),
 ('AFD1', 'EPE'),
 ('AFD1', 'EPS2'),
 ('AFD1', 'EPS3'),
 ('AFD1', 'EVA'),
 ('AFD1', 'E

In [280]:
devset_SFL_filtered['ID_details'] == 'AFD1'+'-FS02_ASR_track2_dev_0716'

0        False
1        False
2        False
3        False
4        False
         ...  
15195    False
15196    False
15197    False
15198    False
15199    False
Name: ID_details, Length: 15200, dtype: bool

In [281]:
Id1 = devset_SFL_filtered['ID_details'][0].split('-')
Id1

['AFD1', 'FS02_ASR_track2_dev_4538']

In [282]:
Id1 = devset_SFL_filtered['ID_details'][0].split('-')[0]
uttrance = devset_SFL_filtered['ID_details'][i].split('-')[1]
Id1+'-'+uttrance


'AFD1-FS02_ASR_track2_dev_6624'

In [283]:
#pairs of vectors
# location of files /net/home/bibash_01/my_project/fearless/fearless/diary/Files_folder

In [284]:
import os

In [24]:
curDir='/net/home/bibash_01/my_project/fearless/fearless/diary/'
filesDir = '/net/home/bibash_01/my_project/fearless/fearless/diary/Files_folder/'

In [ ]:
with open("same_speaker",'w',encoding = 'utf-8') as f:
    os.chdir(filesDir)
        for i in range(len(devset)): 
            x=tempDf.iloc[i]['audio_idprefix']+"  "+tempDf.iloc[i]['transcription']
            f.write(x+'\n')
        f.close()
    os.chdir(curDir)